In [5]:
import ray
from ray import tune
from ray.rllib.env import PettingZooEnv
# from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.models import ModelCatalog
from ray.tune.registry import register_env
from environment import skyjo_env  # Update with your environment module

# skyjo_env_cfg = {"num_players": 3, "render_mode":"human"}

# Register the PettingZoo environment with RLlib
def env_creator(config):
    return skyjo_env.env(**config)

register_env("SkyjoEnv", env_creator)

# RLlib configuration for PPO
config = {
    "env": "SkyjoEnv",
    "env_config": {  # Configuration to initialize the Skyjo environment
        "num_players": 3,
        "score_penalty": 2.0,
        "observe_other_player_indirect": True,
        "mean_reward": 1.0,
        "reward_refunded": 0.001,
        "render_mode": None,
    },
    "multiagent": {
        "policies": {
            i: (None, skyjo_env.SimpleSkyjoEnv().observation_space(i),
                           skyjo_env.SimpleSkyjoEnv().action_space(i), {})
            for i in range(2)  # Adjust based on num_players
        },
        "policy_mapping_fn": lambda agent_id: agent_id,  # Map each agent to its own policy
    },
    "framework": "torch",  # Use Torch or TensorFlow
    "num_workers": 1,  # Adjust for your compute setup
    "train_batch_size": 4000,
    "rollout_fragment_length": 200,
    "sgd_minibatch_size": 128,
    "num_sgd_iter": 10,
    "lr": 1e-4,
    "model": {
        "fcnet_hiddens": [256, 256],
        "fcnet_activation": "relu",
    },
}


ray.init(ignore_reinit_error=True)

# Start training with PPO
analysis = tune.run(
    "PPO",
    config=config,
    stop={"training_iteration": 50},  # Adjust as needed
    checkpoint_freq=10,
)

# Print training results
print("Training completed. Best checkpoint is located at:", analysis.best_checkpoint)

2024-12-02 20:59:58,072	INFO worker.py:1786 -- Started a local Ray instance.
2024-12-02 20:59:58,812	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
/home/henry/Documents/SharedDocuments/Uni/TU/3.Semester/AdvRL/SkyjoAI/.venv/lib/python3.11/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/henry/Documents/SharedDocuments/Uni/TU/3.Semester/AdvRL/SkyjoAI/.venv/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/home/henry/Documents/SharedDocuments/Uni/TU/3.Semester/AdvRL/SkyjoAI/.venv/lib/python3.11/site-package

ValueError: PolicyID `0` not valid! IDs must be a non-empty string, must not contain characters that are also disallowed file- or directory names on Unix/Windows and must not end with a dot `.` or a space ` `.

In [1]:
# Import necessary libraries
import ray
from ray import tune
from ray.rllib.env import PettingZooEnv
from ray.tune.registry import register_env
from pettingzoo import ParallelEnv
import numpy as np
from environment import skyjo_env

skyjo_env_cfg = {"num_players": 3, "render_mode":"human"}

# Step 2: Register the Environment
def env_creator(env_config):
    return skyjo_env.env(**env_config)

register_env("pettingzoo_skyjo", lambda config: PettingZooEnv(env_creator(config)))

# Step 3: Initialize Ray and Configure PPO
ray.init()

# ppo_config = {
#     "env": "pettingzoo_skyjo",
#     "multiagent": {
#         "policies": {
#             "shared_policy": (None, np.array([1]), 2, {})  # Custom policy setup
#         },
#         "policy_mapping_fn": (lambda agent_id, episode, **kwargs: "shared_policy"),
#     },
#     "framework": "torch",  # Can also use "tf" or "tf2"
#     "num_workers": 1,  # Number of parallel workers
# }
env = PettingZooEnv(env_creator(skyjo_env_cfg))
custom_config={
    "env":"pettingzoo_skyjo",
    "model": {
        "custom_model": "pa_model2",
    },
    "framework": "torch",
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
    "num_gpus": 1,
    "num_workers": 4, #os.cpu_count() - 1,
    "num_gpus_per_worker": 0.25,
    "multiagent":{
            "policies": {
                name: (None, env.observation_space, env.action_space, {}) for name in env.agents
            },
            "policy_mapping_fn": lambda agent_id: agent_id
    }
}

# Step 4: Train the Model Using Tune
results = tune.run(
    "PPO",
    config=custom_config,
    stop={"training_iteration": 50},  # Adjust as needed
    checkpoint_freq=10,
)

# Step 5: Shut Down Ray
ray.shutdown()

2024-12-02 22:00:33,470	INFO worker.py:1786 -- Started a local Ray instance.


Obs shape: (55,), type: <class 'tuple'>
Action mask shape: (26,), type: <class 'tuple'>
Observation space for agent 0: Dict('action_mask': Box(0, 1, (26,), int8), 'observations': Box(-24, 127, (55,), int8))
Obs shape: (55,), type: <class 'tuple'>
Action mask shape: (26,), type: <class 'tuple'>
Observation space for agent 1: Dict('action_mask': Box(0, 1, (26,), int8), 'observations': Box(-24, 127, (55,), int8))
Obs shape: (55,), type: <class 'tuple'>
Action mask shape: (26,), type: <class 'tuple'>
Observation space for agent 2: Dict('action_mask': Box(0, 1, (26,), int8), 'observations': Box(-24, 127, (55,), int8))


AssertionError: Dict space element is not an instance of Space: key='0', space=Dict('action_mask': Box(0, 1, (26,), int8), 'observations': Box(-24, 127, (55,), int8))